<a href="https://colab.research.google.com/github/SoonerTuran/DNVA/blob/main/DNVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class ALU(nn.Module):
    def __init__(self, input_size, opcode_size, output_size, flag_size, hidden_size=32):
        super(ALU, self).__init__()

        self.input_size = input_size
        self.opcode_size = opcode_size
        self.output_size = output_size
        self.flag_size = flag_size

        # Layers definition
        self.linear1 = nn.Linear(self.input_size + self.opcode_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, self.output_size + flag_size)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        return x

In [ ]:
import torch
import torch.nn as nn
import math

class Decoder(nn.Module):
    def __init__(self, input_size):
        super(Decoder, self).__init__()

        # Giriş boyutu input_size olarak alınır.
        # Çıkış boyutu ise 2^input_size olarak belirlenir.
        output_size = 2 ** input_size

        # 5 katmanlı bir fully connected (dense) ağ tanımlanıyor.
        self.fc1 = nn.Linear(input_size, input_size*2)
        self.fc2 = nn.Linear(input_size*2, input_size*3)
        self.fc3 = nn.Linear(input_size*3, input_size*4)
        self.fc4 = nn.Linear(input_size*4, input_size*4)
        self.fc5 = nn.Linear(input_size*4, output_size)


    def forward(self, x):
      x = torch.sin(self.fc1(x))
      x = torch.sin(self.fc2(x))
      x = torch.sin(self.fc3(x))
      x = torch.sin(self.fc4(x))
      x = torch.softmax(self.fc5(x), dim=1)
      return x


In [ ]:
import torch
import itertools

input_size = 10
num_samples = 2 ** input_size
output_size = 2 ** input_size

# Tüm olası ikili sayıları oluşturmak için itertools.product kullanılıyor.
binary_values = list(itertools.product([0, 1], repeat=input_size))
# Toplam ikili sayı sayısı num_samples'tan fazla ise, sadece ilk num_samples kadarını alırız.
binary_values = binary_values[:num_samples]

# Liste halindeki ikili sayıları tensörlere dönüştürme
train_inputs = torch.tensor(binary_values, dtype=torch.int, device=device , requires_grad=False)
test_inputs = torch.randint(0, 2, (num_samples, input_size), device=device, requires_grad=False)

# One-hot çıktıları oluşturma
train_outputs = torch.zeros(num_samples, output_size, device=device)
test_outputs = torch.zeros(num_samples, output_size, device=device)
for i in range(num_samples):
    # Her ikili sayıyı ondalık olarak dönüştürme
    decimal1 = int(''.join(map(str, train_inputs[i].tolist())), 2)
    decimal2 = int(''.join(map(str, test_inputs[i].tolist())), 2)
    # Karşılık gelen indekse 1 atama
    train_outputs[i][decimal1] = 1
    test_outputs[i][decimal2] = 1

train_inputs = train_inputs.float()
test_inputs = test_inputs.float()
train_outputs = train_outputs.float()
test_outputs = test_outputs.float()

del binary_values
torch.cuda.empty_cache()

In [ ]:
model = Decoder(input_size=input_size)
model.to(device)

# Setup a loss function
loss_fn = nn.CrossEntropyLoss()

# Setup an optimizer (stochastic gradient descent)
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

In [ ]:
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred.data, 1)
    correct = (predicted == torch.argmax(y_true, 1)).sum().item()
    accuracy = correct / y_true.size(0)
    return accuracy


epochs = 20000

# Track different values
epoch_count = []
loss_values = []
test_loss_values = []
accuracy_values = []
test_accuracy_values = []

#Training
for epoch in range(epochs):
  # Set the model to training mode
  model.train() # train mode in PyTorch sets all parameters that require gradients to require gradients

  # 1. Forward pass
  y_pred = model(train_inputs)

  # 2. Calculate the loss
  loss = loss_fn(y_pred, train_outputs)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Perform backpropagation on the loss with respect to the parameters of the model (calculate gradients of each parameter)
  loss.backward()

  # 5. Step the optimizer (perform gradient descent)
  optimizer.step() # by default how the optimizer changes will acculumate through the loop so... we have to zero them above in step 3 for the next iteration of the loop

  # Calculate training accuracy
  accuracy = calculate_accuracy(y_pred, train_outputs)

  ### Testing
  model.eval() # turns off different settings in the model not needed for evaluation/testing (dropout/batch norm layers)
  with torch.inference_mode(): # turns off gradient tracking & a couple more things behind the scenes
    # 1. Do the forward pass
    test_pred = model(test_inputs)

    # 2. Calculate the loss
    test_loss = loss_fn(test_pred, test_outputs)

    # Calculate testing accuracy
    test_accuracy = calculate_accuracy(test_pred, test_outputs)

  # Print out what's happenin'
  if epoch % 100  == 0:
    epoch_count.append(epoch)
    loss_values.append(loss.detach().item())
    test_loss_values.append(test_loss.detach().item())

    accuracy_values.append(accuracy)
    test_accuracy_values.append(test_accuracy)

    print(f"Epoch: {epoch} | Loss: {loss.detach().item():.6f} | Test loss: {test_loss.detach().item():.6f} | Train Accuracy: {accuracy:.6f} | Test Accuracy: {test_accuracy:.6f}")

  del loss, test_loss, y_pred
  torch.cuda.empty_cache()



In [ ]:
# Plot the loss curves
plt.plot(epoch_count, np.array(torch.tensor(loss_values)), label="Train loss")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values)), label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

# Plot the loss curves
plt.plot(epoch_count, np.array(torch.tensor(accuracy_values)), label="Train Acc")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values)), label="Test Acc")
plt.title("Training and test acc curves")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.inference_mode():
  test_pred = model(test_inputs)

index = torch.randint(low=0, high=512, size=(1,1))[0][0]
print(test_inputs[index])

print(test_pred[index].round(decimals=2))
print(test_pred[index].round(decimals=2).argmax())

# Binary tensoru bir stringe dönüştürelim
binary_str = ''.join([str(int(i)) for i in test_inputs[index]])

# Binary stringi desimal sayıya dönüştürelim
decimal_number = int(binary_str, 2)
print(decimal_number)